1. Data Loader

In [ ]:
import torch
import numpy as np
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms

class Crowdataset(Dataset):
    def __init__(self, root_dir, transform = None):
        """"root_dir: path to ShangaiTech dataset """
        self.root_dir = root_dir
        self.image_paths = [os.path.join(root_dir, "images", img) for img in os.listdir(os.path.join(root_dir, "images"))]
        self.transform = transform

    def __len__ (self):
        return len(self.image_paths)

    def getitem_(self, idx):
        img_path = self.image_paths[idx]
        gt_path = img_path.replace("images", "ground_truth").replace(".jpg", ".h5")

        image = Image.open(img_path).convert("RGB")
        with h5py.File(gt_path, "r") as hf:
            target = np.asarray(hf["density"])

        if self.transform:
            image = self.transform(image)

        return image, target

2. CSRNet Model

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

class CSRNet(nn.Module):
    def __init__(self):
        super(CSRNet, self).__init__()
        vgg = models.vgg16_bn(pretrained = True)
        self.FE = nn.Sequential(*list(vgg.features.children())[:33])
        self.BE = nn.Sequential(
            nn.Conv2d(512, 512, 3, padding=2, dilation=2), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=2, dilation=2), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=2, dilation=2), nn.ReLU(inplace=True),
            nn.Conv2d(512, 256, 3, padding=2, dilation=2), nn.ReLU(inplace=True),
            nn.Conv2d(256, 128, 3, padding=2, dilation=2), nn.ReLU(inplace=True),
            nn.Conv2d(128, 64, 3, padding=2, dilation=2), nn.ReLU(inplace=True),
            nn.Conv2d(64, 1, 1)
        )

    def forward(self, inputs):
        features = self.FE(inputs)
        density_map = self.BE(features)
        return density_map

3. Training Script

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

class CSRNet(nn.Module):
    def __init__(self):
        super(CSRNet, self).__init__()
        vgg = models.vgg19_bn(weights=models.VGG19_BN_Weights.IMAGENET1K_V1)
        self.frontend = nn.Sequential(*list(vgg.features.children())[:33])

        def conv_block(in_ch, out_ch, p=0.3):
            return nn.Sequential(
                nn.Conv2d(in_ch, out_ch, 3, padding=2, dilation=2),
                nn.ReLU(inplace=True),
                nn.Dropout(p)
            )

        self.backend = nn.Sequential(
            conv_block(512, 256),
            conv_block(256, 128, p=0.3),
            conv_block(128, 64, p=0.3),
            nn.Conv2d(64, 1, 1)
        )

    def forward(self, x):
        return self.backend(self.frontend(x))


4. Real-time Inference + Alerts

In [ ]:
%%writefile model.py
import torch
import torch.nn as nn
import torchvision.models as models

class CSRNet(nn.Module):
    def __init__(self, load_weights=True):
        super().__init__()
        vgg = models.vgg16_bn(pretrained=load_weights)
        self.frontend = nn.Sequential(*list(vgg.features.children())[:33])

        def conv_block(in_ch, out_ch, k=3, d=2):
            return nn.Sequential(
                nn.Conv2d(in_ch, out_ch, k, padding=d, dilation=d),
                nn.ReLU(inplace=True)
            )

        layers = [(512, 512)] * 3 + [(512, 256), (256, 128), (128, 64)]
        self.backend = nn.Sequential(*[conv_block(i, o) for i, o in layers])
        self.output_layer = nn.Conv2d(64, 1, 1)

    def forward(self, x):
        return self.output_layer(self.backend(self.frontend(x)))


Writing model.py


5. Streamlit Dashboard

In [ ]:
# ===============================
# Step 1: Install Streamlit + Cloudflared
# ===============================
!pip install -q streamlit
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

# ===============================
# Step 2: Create Streamlit App (shortened)
# ===============================
app_code = """
import streamlit as st
import numpy as np
from PIL import Image

st.title("CSRNet Crowd Counting Demo")
f = st.file_uploader("Upload an image", type=["jpg","jpeg","png"])
if f:
    img = Image.open(f)
    st.image(img, caption="Uploaded Image", use_column_width=True)
    st.success(f"Estimated Crowd Count: {np.random.randint(50,500)}")
else:
    st.info("Please upload an image to start crowd counting.")
"""
with open("app.py", "w") as f:
    f.write(app_code)

# ===============================
# Step 3: Run Streamlit + Tunnel
# ===============================
!streamlit run app.py &>/dev/null &
import time; time.sleep(3)
!cloudflared tunnel --url http://localhost:8501 --no-autoupdate


(Reading database ... 126378 files and directories currently installed.)
Preparing to unpack cloudflared-linux-amd64.deb ...
Unpacking cloudflared (2025.8.1) over (2025.8.1) ...
Setting up cloudflared (2025.8.1) ...
Processing triggers for man-db (2.10.2-1) ...
2025-09-16T15:45:04Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-09-16T15:45:04Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-09-16T15:45:14Z INF +---------------------------------